In [9]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
import Levenshtein
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
#look for duplicates and remove them from original dataset
df_benign_fake = pd.read_csv("./checkpoints/URL_LSTM/21-07-2021_17-51/Benign_Samples_21-07-2021_20-39.csv", names=['url'], squeeze=True)
print("Benign Fake Duplicates: "+ str(df_benign_fake.value_counts()))
df_benign_fake = df_benign_fake.drop_duplicates(keep='first')
print("Benign Fake: " + str(df_benign_fake.shape))
df_benign_fake = pd.concat([df_benign_fake, pd.DataFrame(np.zeros_like(df_benign_fake), columns=["real"])], axis=1)

df_benign_real = pd.read_csv("./Datasets/URL/Benign_list_big_final.csv", names=['url'], squeeze=True)
print("Benign Real Duplicates: "+ str(df_benign_real.value_counts()))
df_benign_real = df_benign_real.drop_duplicates(keep='first')
print("Benign Real : " + str(df_benign_real.shape))
df_benign_real = pd.concat([df_benign_real, pd.DataFrame(np.ones_like(df_benign_real), columns=["real"])], axis=1)

df_spam_fake = pd.read_csv("./checkpoints/URL_LSTM/21-07-2021_20-56/Spam_Samples_21-07-2021_23-07.csv", names=['url'], squeeze=True)
print("Spam Fake Duplicates: "+ str(df_spam_fake.value_counts()))
df_spam_fake = df_spam_fake.drop_duplicates(keep='first')
print("Spam Fake: " + str(df_spam_fake.shape))
df_spam_fake = pd.concat([df_spam_fake, pd.DataFrame(np.zeros_like(df_spam_fake), columns=["real"])], axis=1)

df_spam_real=pd.read_csv("./Datasets/URL/spam_dataset.csv", names=['url'], squeeze=True)
print("Spam Real Duplicates: "+ str(df_spam_real.value_counts()))
df_spam_real = df_spam_real.drop_duplicates(keep='first')
print("Spam Real : " + str(df_spam_real.shape))
df_spam_real = pd.concat([df_spam_real, pd.DataFrame(np.ones_like(df_spam_real), columns=["real"])], axis=1)

Benign Fake Duplicates: http://                                                                                                               28
http://censor.net.ua/jokes/all/page/1/hero/0/category/0/interval/5/author/0/sortby/date                               13
http://allegro.pl/listing/listing.php?bmatch=seng-v10-p-sm-pers-isqm-chl-moda-0414&order=m&p=2&string=%7Bstring%7D     7
http://syosetu.com/searchuser/search/index.php?name1st=%E3%81%88&all=1&all2=1&all3=1&all4=1&p=10                       6
http://syosetu.com/searchuser/search/index.php?name1st=%E3%81%9F&all=1&all2=1&all3=1&all4=1&p=10                       6
                                                                                                                      ..
http://sda.gov.br/interval/arvatiten/ranking.html?src=home_feet                                                        1
http://dantri.com.vn/gits/develop-to-bat-o6-khoc-say-/6-that-voi-tre-ngan-20141105235311513.chn                        1
http://s

In [3]:
#concatenate the real and fake datasets and see if they contain any duplicate addresses (if the algorithm is just copying the data)
df_benign_real_fake=pd.concat([df_benign_real, df_benign_fake], axis=0)
print(df_benign_real_fake[["url"]].value_counts())
print(df_benign_real_fake.shape)
#drop any duplicates and dont keep either of them since any duplicates in the original datasets will have been dropped in the last step
#the only things that will be dropped now are urls that appear in both the real data and fake data (copies)
df_benign_real_fake = df_benign_real_fake.drop_duplicates(subset=['url'], keep=False)
print(df_benign_real_fake.shape)
print((df_benign_real_fake[["real"]].value_counts()))


df_spam_real_fake=pd.concat([df_spam_real, df_spam_fake], axis=0)
print(df_spam_real_fake[["url"]].value_counts())
print(df_spam_real_fake.shape)

df_spam_real_fake = df_spam_real_fake.drop_duplicates(subset=['url'], keep=False)
print(df_spam_real_fake.shape)
print((df_spam_real_fake[["real"]].value_counts()))

url                                                                                                                       
http://syosetu.com/searchuser/search/index.php?name1st=%E3%81%8F&all=1&all2=1&all3=1&all4=1&p=2                               2
http://olx.ua/uk/list/q-%D0%B2%D0%B5%D0%BB%D0%BE%D1%81%D0%B8%D0%BF%D0%B5%D0%B4%D1%8B/?page=3                                  2
http://syosetu.com/searchuser/search/index.php?name1st=%E3%82%8F&all=1&all2=1&all3=1&all4=1&p=10                              2
http://olx.ua/uk/list/q-%D0%B2%D0%B5%D0%BB%D0%BE%D1%81%D0%B8%D0%BF%D0%B5%D0%B4%D1%8B/?search%5Bprivate_business%5D=private    2
http://syosetu.com/searchuser/search/index.php?name1st=%E3%82%8F&all=1&all2=1&all3=1&all4=1&p=2                               2
                                                                                                                             ..
http://hollywoodlife.com/2015/03/29/iggy-azalea-iheartradio-music-awards-outfit-2015-pics-braless-mesh-top/  

In both cases only about 1000ish rows were dropped indicating that there would have been 500 pairs from the real and fake data
this isnt terrible and with more complex data I expect this to be lower


